In [36]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [81]:
df = pd.read_parquet("df_ImparSQL.parquet", engine="pyarrow")

In [ ]:
df_Buroug = pd.read_csv("yellow_t_2018-01.parquet", sep=" , ")

yellow_t_2018-01.parquet

In [82]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan,2018-01-03,2018-01-03,-7.3,-13.5,0.0,0.0,0.33,Clear
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan,2018-01-03,2018-01-03,-7.3,-13.5,0.0,0.0,0.33,Clear
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan,2018-01-03,2018-01-03,-7.3,-13.5,0.0,0.0,0.33,Clear
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan,2018-01-03,2018-01-03,-7.3,-13.5,0.0,0.0,0.33,Clear
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan,2018-01-03,2018-01-03,-7.3,-13.5,0.0,0.0,0.33,Clear


In [38]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,NaN
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30,NaN,NaN
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30,NaN,NaN
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80,NaN,NaN
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,NaN,NaN


In [39]:
"""
def get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):
    labels,values,percent = list(),list(),list()
    if df.isna().sum().sum()>0:
        for column in df.columns:
            if df[column].isna().sum():
                labels.append(column)
                values.append(df[column].isna().sum())
                percent.append((df[column].isna().sum() / df.shape[0]) * 100)
        #Make a dataframe 
        missings=pd.DataFrame({'Colúmnas':labels,'Valores faltantes':values,'PorcentajePerdido':percent}).sort_values(by='PorcentajePerdido',ascending=False)
        

        if plot:
            plt.figure(figsize=figsize)
            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title('Porcentaje de valores perdidos',size=22)
            for i in plot.patches:
                plot.annotate(i.get_height(),
                (i.get_x() + i.get_width() / 2, i.get_height()),
                ha = 'center', va = 'baseline', fontsize = 12,
                color = 'black', xytext = (0,1),
                textcoords = 'offset pixels')
            locs, labels = plt.xticks()
            plt.setp(labels, rotation=rotation,size=16)
            plt.xlabel('Colúmnas',size=18)
            plt.ylabel('Porcentaje',size=18)
            plot.set_title('Porcentaje de valores perdidos',size=18)

        return missings
    else:
        return False
        """

'\ndef get_missings(df,plot=True,rotation=45,figsize=(10,5), * kwargs):\n    labels,values,percent = list(),list(),list()\n    if df.isna().sum().sum()>0:\n        for column in df.columns:\n            if df[column].isna().sum():\n                labels.append(column)\n                values.append(df[column].isna().sum())\n                percent.append((df[column].isna().sum() / df.shape[0]) * 100)\n        #Make a dataframe \n        missings=pd.DataFrame({\'Colúmnas\':labels,\'Valores faltantes\':values,\'PorcentajePerdido\':percent}).sort_values(by=\'PorcentajePerdido\',ascending=False)\n        \n\n        if plot:\n            plt.figure(figsize=figsize)\n            plot = sns.barplot(x=missings.Colúmnas,y=round(missings.PorcentajePerdido, 2), palette = "Blues_r")# .set_title(\'Porcentaje de valores perdidos\',size=22)\n            for i in plot.patches:\n                plot.annotate(i.get_height(),\n                (i.get_x() + i.get_width() / 2, i.get_height()),\n          

In [40]:
"""get_missings(df)"""

'get_missings(df)'

Eliminamos las columnas que consideramos insignificantes

In [41]:
df.drop(columns=["congestion_surcharge","airport_fee"], inplace=True)

Dividir df

In [42]:
df.head(1)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8


In [43]:
df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)

In [44]:
df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']

In [45]:
df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']

In [46]:

df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
mask = df.dayofmonth % 3 == 0
df_Impar =df[mask]
df_Impar = df_Impar.copy()
#intento de liberar ram
df = "a"
mask = "b"



In [47]:
df_Impar.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
644899,2,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,N,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3
599145,2,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,N,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3
480025,1,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,N,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3
478103,1,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,N,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3
469218,2,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,N,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3


In [48]:
#df_Impar.tpep_pickup_datetime.dt.floor("H")

De ahora en mas df_Impar es el df a usar

In [49]:
df_Impar.shape

(2908992, 18)

In [50]:
df_Impar.drop(columns=["VendorID","store_and_fwd_flag"], inplace=True)

vamos a obtener los borous de salida y de llegada.

In [51]:
#se carga el csv con las zonas
zona = pd.read_csv("taxi+_zone_lookup.csv")
#se borran las columnas que no usamos
zona.drop(columns=["Zone","service_zone"], inplace=True)
#cambiamos el nombre para poder hacer el merge
zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
#se hace el merge y se crea la columnna resultante del merge "Borough"
df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
#se le cambia el nombre a "boroug" para que haga referencia al borough de partida
df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
#mismo procedimiento para borough de llegada
zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
zona = "A"

In [52]:
df_Impar.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,230,162,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,88,230,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,68,249,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,138,263,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,170,263,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan


Trabajamos con el clima

In [53]:
clima = pd.read_csv("new york 2018-01-01 to 2018-02-01.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]

In [54]:
clima = clima[clima["datetime"] >= '2018-01-01 00:00:00']
clima = clima[clima["datetime"] <= '2018-01-31 00:00:00']

In [55]:
clima.head()

,datetime,temp,feelslike,precip,snow,snowdepth,conditions
0,2018-01-01 00:00:00,-12.6,-20.8,0.0,0.0,0.68,Clear
1,2018-01-01 01:00:00,-12.6,-20.9,0.0,0.0,0.60,Clear
2,2018-01-01 02:00:00,-12.7,-21.1,0.0,0.0,0.51,Clear
3,2018-01-01 03:00:00,-13.2,-21.2,0.0,0.0,0.43,Clear
4,2018-01-01 04:00:00,-13.3,-20.9,0.0,0.0,0.34,Clear


creamos columna auxiliar de hora

In [56]:
df_Impar["hora"]= df_Impar.tpep_pickup_datetime.dt.floor("H")
clima.datetime = clima['datetime'].apply(pd.to_datetime)
clima["hora"]= clima['datetime'].dt.floor("H")

In [57]:
#cambiamos el nombre para poder hacer el merge
#zona.rename(columns={"datetime": "tpep_pickup_datetime"},inplace=True)

In [58]:
df_Impar.shape

(2908992, 19)

se cargaron los datos del clima al dataframe.

In [59]:
df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

In [60]:
dfsinceros=df_Impar[df_Impar.fare_amount != 0]
dfsinceros=dfsinceros[dfsinceros.fare_amount>0]
dfsinceros = dfsinceros[dfsinceros.trip_distance != 0]

In [61]:
df1=dfsinceros[dfsinceros.RatecodeID == 1]
df2=dfsinceros[dfsinceros.RatecodeID == 2]
df3=dfsinceros[dfsinceros.RatecodeID == 3]
df4=dfsinceros[dfsinceros.RatecodeID == 4]
df5=dfsinceros[dfsinceros.RatecodeID == 5]

In [62]:
def millas(df):
  df['km_dollar'] = df['trip_distance']/df['fare_amount']
  return df
  

In [63]:
df1 = millas(df1)
df2 = millas(df2)
df3 = millas(df3)
df4 = millas(df4)
df5 = millas(df5)

In [64]:
avg_km1 =df1['km_dollar'].mean()
avg_km2 =df2['km_dollar'].mean()
avg_km3 =df3['km_dollar'].mean()
avg_km4 =df4['km_dollar'].mean()
avg_km5 =df5['km_dollar'].mean()

In [65]:
df_Impar.shape

(2908992, 26)

se borran los que tienen cero en trip distance y fare amount

In [66]:

df_Impar["borrar"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]==0  and row["trip_distance"]==0 else 0, axis=1)
df_Impar = df_Impar.drop(df_Impar[df_Impar.borrar == 1].index)
df_Impar = df_Impar.drop(columns="borrar")

In [67]:
df_Impar.shape

(2908638, 26)

In [68]:
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km1 if row["RatecodeID"]==1  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km2 if row["RatecodeID"]==2  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km3 if row["RatecodeID"]==3  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km4 if row["RatecodeID"]==4  and row["trip_distance"]==0 else row["trip_distance"], axis=1)
df_Impar["trip_distance"] = df_Impar.apply(lambda row: row["fare_amount"]*avg_km5 if row["RatecodeID"]==5  and row["trip_distance"]==0 else row["trip_distance"], axis=1)


Hasta ahora, se borraron las filas con trip distance y fare amount en cero, y se imputo la distancia recorrida en los campos correspondientes segun su Ratecode ID. restaria borrar los Ratecode ID 6 y 99 que son practicamente irrelevantes.

vamos a revisar y decidir que hacer con los precios negativos

Generamos csv para tablas sql

In [69]:
#df_Impar.to_csv('df_ImparSQL.csv', sep=',')

In [70]:
df_Impar[df_Impar.fare_amount<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
43,2018-01-03 00:00:27,2018-01-03 00:01:48,5,0.240000,1,79,79,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
6370,2018-01-03 01:37:09,2018-01-03 01:37:39,1,0.010000,1,132,132,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,3,Queens,Queens,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
6919,2018-01-03 01:52:56,2018-01-03 01:54:08,2,0.470000,1,48,48,3,-3.5,-0.5,-0.5,0.0,0.0,-0.3,-4.8,3,Manhattan,Manhattan,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
8010,2018-01-03 02:31:10,2018-01-03 02:33:12,2,0.450000,1,263,263,3,-4.0,-0.5,-0.5,0.0,0.0,-0.3,-5.3,3,Manhattan,Manhattan,2018-01-03 02:00:00,2018-01-03 02:00:00,-7.4,-12.4,0.0,0.0,0.25,Clear


In [71]:
df_Impar[df_Impar.trip_distance<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
11905,2018-01-03 05:24:43,2018-01-03 05:25:19,1,-732.619596,5,48,48,3,-34.8,0.0,-0.5,0.0,0.0,-0.3,-35.6,3,Manhattan,Manhattan,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
12080,2018-01-03 05:29:10,2018-01-03 05:29:53,1,-4.954313,3,129,129,4,-20.0,-0.5,0.0,0.0,0.0,-0.3,-20.8,3,Queens,Queens,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
13177,2018-01-03 05:49:31,2018-01-03 05:50:42,4,-0.614998,1,230,230,4,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 05:00:00,2018-01-03 05:00:00,-8.6,-13.6,0.0,0.0,0.13,Partially cloudy
19874,2018-01-03 06:50:34,2018-01-03 06:50:42,2,-0.512498,1,132,132,4,-2.5,0.0,-0.5,0.0,0.0,-0.3,-3.3,3,Queens,Queens,2018-01-03 06:00:00,2018-01-03 06:00:00,-8.5,-8.5,0.0,0.0,0.08,Partially cloudy


In [72]:
df_Impar[df_Impar.total_amount<0].head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
43,2018-01-03 00:00:27,2018-01-03 00:01:48,5,0.240000,1,79,79,3,-3.0,-0.5,-0.5,0.0,0.0,-0.3,-4.3,3,Manhattan,Manhattan,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
3705,2018-01-03 00:46:34,2018-01-03 00:46:42,1,-1578.921543,5,97,97,3,-75.0,0.0,-0.5,0.0,0.0,-0.3,-75.8,3,Brooklyn,Brooklyn,2018-01-03 00:00:00,2018-01-03 00:00:00,-7.3,-13.5,0.0,0.0,0.33,Clear
6370,2018-01-03 01:37:09,2018-01-03 01:37:39,1,0.010000,1,132,132,3,-2.5,-0.5,-0.5,0.0,0.0,-0.3,-3.8,3,Queens,Queens,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
6919,2018-01-03 01:52:56,2018-01-03 01:54:08,2,0.470000,1,48,48,3,-3.5,-0.5,-0.5,0.0,0.0,-0.3,-4.8,3,Manhattan,Manhattan,2018-01-03 01:00:00,2018-01-03 01:00:00,-7.7,-13.2,0.0,0.0,0.29,Clear
8010,2018-01-03 02:31:10,2018-01-03 02:33:12,2,0.450000,1,263,263,3,-4.0,-0.5,-0.5,0.0,0.0,-0.3,-5.3,3,Manhattan,Manhattan,2018-01-03 02:00:00,2018-01-03 02:00:00,-7.4,-12.4,0.0,0.0,0.25,Clear


In [73]:
df_Impar.loc[df_Impar['tpep_pickup_datetime'] == "2018-01-27 10:59:14"]

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,hora,datetime,temp,feelslike,precip,snow,snowdepth,conditions
2390960,2018-01-27 10:59:14,2018-01-27 11:20:14,1,1.00,1,234,186,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390961,2018-01-27 10:59:14,2018-01-27 11:07:50,1,1.70,1,236,151,1,8.5,0.0,0.5,0.00,0.0,0.3,9.30,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390962,2018-01-27 10:59:14,2018-01-27 11:21:12,1,4.35,1,236,246,2,18.0,0.0,0.5,0.00,0.0,0.3,18.80,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390963,2018-01-27 10:59:14,2018-01-27 11:21:08,1,3.34,1,148,246,1,16.0,0.0,0.5,3.36,0.0,0.3,20.16,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390964,2018-01-27 10:59:14,2018-01-27 11:02:01,1,0.40,1,262,263,1,4.0,0.0,0.5,0.95,0.0,0.3,5.75,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390965,2018-01-27 10:59:14,2018-01-27 11:10:44,1,2.77,1,48,43,2,11.0,0.0,0.5,0.00,0.0,0.3,11.80,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390966,2018-01-27 10:59:14,2018-01-27 11:06:16,1,1.61,1,237,170,1,7.5,0.0,0.5,1.66,0.0,0.3,9.96,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390967,2018-01-27 10:59:14,2018-01-27 11:01:22,1,0.77,1,237,236,2,4.0,0.0,0.5,0.00,0.0,0.3,4.80,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390968,2018-01-27 10:59:14,2018-01-27 11:09:15,3,1.01,1,90,68,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy
2390969,2018-01-27 10:59:14,2018-01-27 11:08:39,1,1.50,99,238,142,1,8.5,0.0,0.5,1.86,0.0,0.3,11.16,27,Manhattan,Manhattan,2018-01-27 10:00:00,2018-01-27 10:00:00,5.1,3.0,0.0,0.0,0.0,Partially cloudy


In [74]:
df_Impar.RatecodeID.unique()

array([ 1,  2,  5,  4,  3, 99,  6], dtype=int64)

In [75]:
df_Impar.drop(columns=["PULocationID","DOLocationID","datetime","hora"], inplace=True)
#df_Impar.drop(columns=["PULocationID","DOLocationID","datetime","hora"], inplace=True)

In [76]:
#el dataframe sin 0s
df_Impar['km_dollar'] = df_Impar['trip_distance']/df_Impar['fare_amount']

avg_km = df_Impar['km_dollar'].mean()

km_ride = df_Impar['trip_distance']


In [77]:
round(avg_km,4)

inf

In [78]:

price=df_Impar['fare_amount']

df_Impar['trip_distance']=df_Impar['trip_distance'].replace(to_replace = 0, value = df_Impar['km_dollar'].mean()*df_Impar['fare_amount'], inplace= True)


ValueError: Series.replace cannot use dict-value and non-None to_replace

In [ ]:
df_Impar.head(1)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,km_dollar
0,2018-01-03,2018-01-03,2,1.53,1,2,10.5,1.0,0.5,0.0,0.0,0.3,12.3,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.145714


In [ ]:
dfRatecode=df_Impar.groupby("RatecodeID").agg({'trip_distance':'sum'})

In [ ]:
df_count = df_Impar.RatecodeID.value_counts()


In [ ]:
df_count

1     2835467
2       58056
5        8375
3        4710
4        1618
99         32
6          13
Name: RatecodeID, dtype: int64

In [ ]:

df_count = pd.DataFrame(df_count, columns=['RatecodeID'])


In [ ]:
df_count.reset_index(inplace=True)
df_count.rename(columns={"index": "RatecodeID","RatecodeID":"Cantidad"},inplace=True)


In [ ]:
df_count.head(7)

,RatecodeID,Cantidad
0,1,2835467
1,2,58056
2,5,8375
3,3,4710
4,4,1618
5,99,32
6,6,13


In [ ]:
dfRatecode = pd.merge(dfRatecode,df_count, how="left", on=["RatecodeID"])

In [ ]:
dfRatecode.head()

,RatecodeID,trip_distance,Cantidad
0,1,7070382.33,2835467
1,2,987081.95,58056
2,3,77967.88,4710
3,4,30734.76,1618
4,5,49585.75,8375


In [ ]:
dfRatecode["km_dollar"] = dfRatecode.trip_distance/dfRatecode.Cantidad

AttributeError: 'DataFrame' object has no attribute 'fare_amount'

In [ ]:
dfRatecode.head(10)

,RatecodeID,fare_amount,Cantidad,km_dollar
0,1,31479383.18,2835467,11.102010
1,2,3008696.50,58056,51.824041
2,3,310009.00,4710,65.819321
3,4,116236.45,1618,71.839586
4,5,529885.79,8375,63.269945
5,6,42.50,13,3.269231
6,99,1312.82,32,41.025625


In [ ]:
df_Impar.head(5)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,km_dollar
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.145714
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.226190
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.163636
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.279245
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.261000


In [ ]:
type(avg_km)

numpy.float64

In [ ]:
df_Impar.head(5)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,km_dollar
0,2018-01-03 00:00:00,2018-01-03 00:00:00,2,1.53,1,2,10.5,1.0,0.5,0.00,0.00,0.3,12.30,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.145714
1,2018-01-03 00:00:00,2018-01-03 00:00:00,1,4.75,1,2,21.0,0.0,0.5,0.00,0.00,0.3,21.80,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.226190
2,2018-01-03 00:00:00,2018-01-03 00:04:33,1,0.90,1,1,5.5,0.5,0.5,1.35,0.00,0.3,8.15,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.163636
3,2018-01-03 00:00:01,2018-01-03 00:21:06,1,7.40,1,1,26.5,0.5,0.5,6.70,5.76,0.3,40.26,3,Queens,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.279245
4,2018-01-03 00:00:02,2018-01-03 00:09:33,1,2.61,1,1,10.0,0.5,0.5,2.26,0.00,0.3,13.56,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear,0.261000


In [ ]:
df_Impar.rename(columns={"tpep_pickup_datetime":"time_pickup","tpep_dropoff_datetime":"time_dropoff"},inplace=True)

In [ ]:
df_Impar.loc[df_Impar['trip_distance'] == 0]


,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,...,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions
85,2018-01-03 00:00:48,2018-01-03 00:00:53,1,0.0,5,1,15.0,0.0,0.5,3.16,...,18.96,3,Unknown,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear
196,2018-01-03 00:01:49,2018-01-03 00:01:49,2,0.0,1,2,31.5,1.0,0.5,0.00,...,39.06,3,Queens,Unknown,-7.3,-13.5,0.0,0.0,0.33,Clear
343,2018-01-03 00:03:23,2018-01-03 00:03:26,1,0.0,5,1,5.0,0.0,0.5,0.00,...,5.80,3,Unknown,Brooklyn,-7.3,-13.5,0.0,0.0,0.33,Clear
354,2018-01-03 00:03:28,2018-01-04 00:03:02,2,0.0,5,1,20.0,0.0,0.0,0.00,...,20.30,3,Queens,Queens,-7.3,-13.5,0.0,0.0,0.33,Clear
516,2018-01-03 00:05:02,2018-01-03 00:05:31,1,0.0,5,1,69.2,0.0,0.0,0.00,...,69.50,3,Brooklyn,Brooklyn,-7.3,-13.5,0.0,0.0,0.33,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908613,2018-01-30 23:57:18,2018-01-30 23:57:44,1,0.0,1,3,2.5,0.5,0.5,0.00,...,3.80,30,Manhattan,Manhattan,-3.6,-8.9,0.0,0.0,0.91,Clear
2908666,2018-01-30 23:57:44,2018-01-30 23:57:48,1,0.0,1,3,2.5,0.5,0.5,0.00,...,3.80,30,Brooklyn,Brooklyn,-3.6,-8.9,0.0,0.0,0.91,Clear
2908754,2018-01-30 23:58:19,2018-01-30 23:58:19,1,0.0,1,2,2.5,0.5,0.5,0.00,...,3.80,30,Manhattan,Unknown,-3.6,-8.9,0.0,0.0,0.91,Clear
2908870,2018-01-30 23:59:06,2018-01-30 23:59:25,1,0.0,5,1,63.0,0.0,0.5,12.76,...,76.56,30,Manhattan,Manhattan,-3.6,-8.9,0.0,0.0,0.91,Clear


In [ ]:
pd.options.display.max_columns=None

In [ ]:
dfAux = df_Impar[df_Impar["trip_distance"] != 0]

yo sumaria todas las distancias de RatecodeID = 1, sumaria todas los precios de fare_amount, calcularia cuanto cuesta una milla.

In [ ]:
df_Impar.head(3)

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions
0,2018-01-03,2018-01-03 00:00:00,2,1.53,1,2,10.5,1.0,0.5,0.00,0.0,0.3,12.30,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear
1,2018-01-03,2018-01-03 00:00:00,1,4.75,1,2,21.0,0.0,0.5,0.00,0.0,0.3,21.80,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear
2,2018-01-03,2018-01-03 00:04:33,1,0.90,1,1,5.5,0.5,0.5,1.35,0.0,0.3,8.15,3,Manhattan,Manhattan,-7.3,-13.5,0.0,0.0,0.33,Clear


In [ ]:
dfAux.passenger_count.max()

9

In [ ]:
dfAux.total_amount.max()

3006.8

In [ ]:
df_Impar.loc[df_Impar['total_amount'] == 3006.8]

,time_pickup,time_dropoff,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions
2091985,2018-01-24 12:39:15,2018-01-24 12:44:43,1,1.2,1,2,3006.0,0.0,0.5,0.0,0.0,0.3,3006.8,24,Queens,Brooklyn,5.2,1.2,0.0,0.0,0.0,Partially cloudy


In [ ]:
dfAux.total_amount.max()

3006.8

In [ ]:
#Tratamiento de Outliers
Q1 = dfAux["trip_distance"].quantile(0.02)
Q3 = dfAux["trip_distance"].quantile(0.98)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 

In [ ]:
#Tratamiento de Outliers
Q1 = dfAux["fare_amount"].quantile(0.02)
Q3 = dfAux["fare_amount"].quantile(0.98)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliersSup = (Q3 + (1.5*IQR)) 

In [ ]:
outliersSup

124.75

In [ ]:
dfAux.shape

(2891059, 22)

In [ ]:
df

In [ ]:
dfAux[dfAux.fare_amount>outliersSup].fare_amount.sort_values(ascending=False)

2091985    3006.0
1033229    2409.0
525866      950.0
999544      557.0
1255936     495.0
            ...  
1088996     125.0
17240       125.0
1732691     125.0
1273405     125.0
2133146     125.0
Name: fare_amount, Length: 794, dtype: float64

In [ ]:
dfAux.trip_distance.sum()

8218524.52

In [ ]:
dfAux.fare_amount.sum()

35048244.270000026

In [ ]:
df2['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[9]


NameError: name 'df2' is not defined

In [ ]:
#data_parcial = data_top_9[data_top_9["Date"] >= '2021-07-01 00:00:00']
data_parcial = df[df["tpep_pickup_datetime"] <= '2018-01-04 00:00:00']
data_parcial = data_parcial[data_parcial["tpep_pickup_datetime"] >= '2018-01-03 23:00:00']

In [ ]:
data_parcial.index[[1]]

Int64Index([722390], dtype='int64')

In [ ]:
data_parcial.reset_index(inplace=True)

In [ ]:
data_parcial.head(5)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
0,722237,2,2018-01-03 23:03:00,2018-01-03 23:13:02,3,1.19,1,N,186,161,2,8.0,0.5,0.5,0.00,0.0,0.3,9.30,3
1,722390,2,2018-01-03 23:01:55,2018-01-03 23:09:39,1,1.13,1,N,234,186,1,7.0,0.5,0.5,2.08,0.0,0.3,10.38,3
2,722753,2,2018-01-03 23:00:46,2018-01-03 23:07:39,2,1.05,1,N,230,163,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80,3
3,723066,2,2018-01-03 23:02:08,2018-01-03 23:04:55,1,1.15,1,N,140,262,2,5.0,0.5,0.5,0.00,0.0,0.3,6.30,3
4,723137,2,2018-01-03 23:00:55,2018-01-03 23:22:04,1,2.99,1,N,113,48,1,15.5,0.5,0.5,1.50,0.0,0.3,18.30,3


In [ ]:
data_parcial.tail(5)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
8379,747843,2,2018-01-03 23:57:07,2018-01-04 00:00:29,1,0.65,1,N,186,68,1,4.5,0.5,0.5,0.08,0.0,0.3,5.88,3
8380,749986,2,2018-01-03 23:17:31,2018-01-03 23:20:56,1,0.62,1,N,211,114,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80,3
8381,749987,2,2018-01-03 23:26:01,2018-01-04 22:41:13,3,4.73,1,N,144,61,2,16.0,0.5,0.5,0.00,0.0,0.3,17.30,3
8382,752203,2,2018-01-03 23:04:11,2018-01-03 23:16:49,2,3.36,1,N,97,89,2,13.0,0.5,0.5,0.00,0.0,0.3,14.30,3
8383,862436,2,2018-01-03 23:52:49,2018-01-04 00:13:28,1,6.02,1,N,261,255,1,21.0,0.5,0.5,4.46,0.0,0.3,26.76,3


In [ ]:
#data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[722237] 

In [ ]:
data_parcial["dayofmonth"] = data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
datafiltrada= data_parcial.head(0)



In [ ]:
datafiltrada

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth


In [ ]:
#data_parcial.loc[data_parcial.index == 148 ]

In [ ]:
#data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[0]

In [ ]:
data_parcial.head(2)

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
0,722237,2,2018-01-03 23:03:00,2018-01-03 23:13:02,3,1.19,1,N,186,161,2,8.0,0.5,0.5,0.00,0.0,0.3,9.30,3
1,722390,2,2018-01-03 23:01:55,2018-01-03 23:09:39,1,1.13,1,N,234,186,1,7.0,0.5,0.5,2.08,0.0,0.3,10.38,3


In [ ]:
#data_parcial= df.head(10)
data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[1]

3

In [ ]:
data_parcial

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
0,722237,2,2018-01-03 23:03:00,2018-01-03 23:13:02,3,1.19,1,N,186,161,2,8.0,0.5,0.5,0.00,0.0,0.3,9.30,3
1,722390,2,2018-01-03 23:01:55,2018-01-03 23:09:39,1,1.13,1,N,234,186,1,7.0,0.5,0.5,2.08,0.0,0.3,10.38,3
2,722753,2,2018-01-03 23:00:46,2018-01-03 23:07:39,2,1.05,1,N,230,163,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80,3
3,723066,2,2018-01-03 23:02:08,2018-01-03 23:04:55,1,1.15,1,N,140,262,2,5.0,0.5,0.5,0.00,0.0,0.3,6.30,3
4,723137,2,2018-01-03 23:00:55,2018-01-03 23:22:04,1,2.99,1,N,113,48,1,15.5,0.5,0.5,1.50,0.0,0.3,18.30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8379,747843,2,2018-01-03 23:57:07,2018-01-04 00:00:29,1,0.65,1,N,186,68,1,4.5,0.5,0.5,0.08,0.0,0.3,5.88,3
8380,749986,2,2018-01-03 23:17:31,2018-01-03 23:20:56,1,0.62,1,N,211,114,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80,3
8381,749987,2,2018-01-03 23:26:01,2018-01-04 22:41:13,3,4.73,1,N,144,61,2,16.0,0.5,0.5,0.00,0.0,0.3,17.30,3
8382,752203,2,2018-01-03 23:04:11,2018-01-03 23:16:49,2,3.36,1,N,97,89,2,13.0,0.5,0.5,0.00,0.0,0.3,14.30,3


In [ ]:
data_parcial.dayofmonth[1]

3

In [ ]:
df.loc[df['PassengerId'] == 148 ]

KeyError: 'PassengerId'

In [ ]:
mask = df.dayofmonth % 3 == 0

In [ ]:
dataFiltrada = data_parcial[mask]

In [ ]:
dataFiltrada.shape

(8383, 19)

In [ ]:
pd.DataFrame([data_parcial.iloc[1]])

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
1,722390,2,2018-01-03 23:01:55,2018-01-03 23:09:39,1,1.13,1,N,234,186,1,7.0,0.5,0.5,2.08,0.0,0.3,10.38,3


In [ ]:
datafiltrada = data_parcial.drop(data_parcial.iloc[2], axis = 0)

KeyError: "[722753, Timestamp('2018-01-03 23:00:46'), Timestamp('2018-01-03 23:07:39'), 1.05, 'N', 6.5, 0.5, 0.5, 0.3, 7.8] not found in axis"

In [ ]:
datafiltrada = pd.concat(pd.DataFrame([data_parcial.iloc[6]]))

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
datafiltrada

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth
6,723505,2,2018-01-03 23:00:08,2018-01-03 23:07:23,1,1.73,1,N,141,163,1,8.0,0.5,0.5,1.5,0.0,0.3,10.8,3


In [ ]:
for i in range(0,data_parcial.shape[0]-1):  
  if data_parcial.dayofmonth[i] % 3 != 0:
    #data_parcial.drop(data_parcial.index[i], axis = 0, inplace=True)
    datafiltrada=pd.concat(data_parcial.iloc[i])


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [ ]:
for i in range(0,data_parcial.shape[0]-1):  
  if data_parcial['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[i+1] % 3 != 0:
    #data_parcial.drop(data_parcial.index[i], axis = 0, inplace=True)
    datafiltrada = datafiltrada.concat(pd.DataFrame([data_parcial.iloc[i]]))


KeyboardInterrupt: 

In [ ]:
#datafiltrada.append(data_parcial.loc[data_parcial.index == 1])

In [ ]:
datafiltrada

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
8313,744788,2,2018-01-03 23:59:39,2018-01-04 00:16:01,2,3.0,1,N,113,163,1,13.0,0.5,0.5,2.86,0.0,0.3,17.16


In [ ]:
df2.apply(
    lambda row: if df2['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day[0] , axis=1).fillna(0)

SyntaxError: invalid syntax (3523998058.py, line 2)

In [ ]:
df.

SyntaxError: invalid syntax (791285630.py, line 1)